#**Chocolate Clustering**

Se usará el dataset [Chocolate Bar Ratings](https://www.kaggle.com/rtatman/chocolate-bar-ratings).

Para realizar el análisis:



1. Leer los datos
2. Analizar los datos
3. Calcular la similaridad entre pares de chocolates
4. Usar k-means para agrupar los chocolates
5. Usar la regla del codo para determinar el número de clusters óptimo



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#1. Lectura del dataset y algo de limpieza

In [ ]:
print("Hola")

Hola


In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

path = "/content/drive/MyDrive/CursoPythonAvanzado/BreveRepasoDePython/Datasets/flavors_of_cacao.csv";

#Puede renombrar las columnas para un manejo más simple
header = ['NameCompany','SpecificBean','REF','ReviewDate','CocoaPercent','CompanyLocation','Rating','BeanType','BroadBeanOrigin']

chocData = pd.read_csv(path, sep=",", encoding='latin-1',names=header)

In [ ]:
chocData = chocData.drop(index=0)
chocData["CocoaPercent"] = (chocData["CocoaPercent"].str.rstrip('%').astype('float'))/100
Rating=chocData.Rating.astype(float)
chocData.Rating = Rating

In [ ]:
chocData

In [ ]:
chocData.isnull().sum()

In [ ]:
#Analice los datos y determine si debe arreglarse algo por ejemplo
#Amsterdam y Holland se refieren al mismo país
#Niacragua y Nicaragua, también, etc.


In [ ]:
#Reemplazar valores vacíos/null con "Blend"
#También revise las abreviaturas
#En su caso, cree una función donde se reemplace una cada una

def changeName(columna,old,new):
  chocData[columna].replace(old,new,inplace=True)

In [ ]:

changeName('CompanyLocation','Amsterdam','Holland')
changeName('CompanyLocation','Niacragua','Nicaragua')
changeName('CompanyLocation','Eucador','Ecuador')
changeName('CompanyLocation','Domincan Republic','Dominican Republic')

changeName('BeanType','Â\xa0','Blend')
changeName('BeanType',np.NaN, ' ')

changeName('BroadBeanOrigin', np.NaN,' ')
changeName('BroadBeanOrigin','Â\xa0','Mexico')
changeName('BroadBeanOrigin','Carribean(DR/Jam/Tri)','Caribbean, DR, Jamaica, Trinidad')
changeName('BroadBeanOrigin','Carribean','Caribbean')
changeName('BroadBeanOrigin','Dom. Rep., Madagascar','DR, Madagascar')
changeName('BroadBeanOrigin','Dominican Republic','DR')
changeName('BroadBeanOrigin','Dominican Rep., Bali','DR, Bali')
changeName('BroadBeanOrigin','Ecuador, Mad., PNG','Ecuador, Madagascar, PNG')
changeName('BroadBeanOrigin','Broad Bean\nOrigin',' ')
changeName('BroadBeanOrigin','Central and S. America','Central, America')
changeName('BroadBeanOrigin','Cost Rica, Ven','Cost Rica, Venezuela')
changeName('BroadBeanOrigin','Ghana & Madagascar','Ghana, Madagascar')
changeName('BroadBeanOrigin','Ghana, Domin. Rep','Ghana, DR')
changeName('BroadBeanOrigin','Gre., PNG, Haw., Haiti, Mad','Greece, PNG, Hawaii, Haiti, Madagascar')
changeName('BroadBeanOrigin','Mad., Java, PNG','Madagascar, Java, PNG')
changeName('BroadBeanOrigin','Madagascar & Ecuador','Madagascar, Ecuador')
changeName('BroadBeanOrigin','Peru(SMartin,Pangoa,nacional)','Peru, SMartin, Pangoa, nacional')
changeName('BroadBeanOrigin','Guat., D.R., Peru, Mad., PNG','Guatemala, DR, Peru, Madagascar')
changeName('BroadBeanOrigin','Peru, Dom. Rep','Peru, DR')
changeName('BroadBeanOrigin','PNG, Vanuatu, Mad','PNG, Vanuatu, Madagascar')
changeName('BroadBeanOrigin','Sao Tome & Principe','Sao Tome, Principe')
changeName('BroadBeanOrigin','Trinidad, Tobago','Trinidad')
changeName('BroadBeanOrigin','Trinidad-Tobago','Trinidad')
changeName('BroadBeanOrigin','Ven, Bolivia, D.R.','Venezuela, Bolivia, DR')
changeName('BroadBeanOrigin','Ven, Trinidad, Ecuador','Venezuela, Trinidad, Ecuador')
changeName('BroadBeanOrigin','Ven., Indonesia, Ecuad.','Venezuela, Indonesia, Ecuador')
changeName('BroadBeanOrigin','Ven., Trinidad, Mad.','Venezuela, Trinidad, Madagascar')
changeName('BroadBeanOrigin','Ven.,Ecu.,Peru,Nic.','Venezuela, Ecuador, Peru, Nicaragua')
changeName('BroadBeanOrigin','Venez,Africa,Brasil,Peru,Mex','Venezuela, Africa, Brasil, Peru, Mexico')
changeName('BroadBeanOrigin','Venezuela, Dom. Rep.','Venezuela, DR')
changeName('BroadBeanOrigin','Venezuela/ Ghana','Venezuela, Ghana')


In [ ]:
list(sorted(chocData.BroadBeanOrigin.unique()))

#2. Analizar los datos

In [ ]:
#Analice las fechas de
sns.histplot(sorted(chocData.ReviewDate));
plt.xticks(rotation=45);

In [ ]:
#Verifique la distribución 'rating', si es una distribución gaussiana
#debería normalizar (chocData['rating']-chocData['rating'].mean())/chocData['rating'].std ()
#sns.distplot(sorted(chocData.Rating),kde=True);
sns.histplot(chocData.Rating,stat='density',kde=True);

In [ ]:
chocData['Rating'] = (chocData['Rating']-chocData['Rating'].mean())/chocData['Rating'].std ();

In [ ]:
#despues de normalizar
sns.histplot(chocData.Rating,stat='density',kde=True);

In [ ]:
#Examine la distribución de 'cocoaPorcentaje'
sns.histplot(sorted(chocData.CocoaPercent),stat='density',kde=True);

In [ ]:
chocData['CocoaPercent'] = (chocData['CocoaPercent']-chocData['CocoaPercent'].mean())/chocData['CocoaPercent'].std ()

In [ ]:
#despues de normalizar
sns.histplot(sorted(chocData.CocoaPercent),stat='density',kde=True);

#3. Calcular la similaridad entre pares de chocholares



Utilice la raíz del error al cuadrado (RMSE) de todas las características numéricas.

Recuerde del ejemplo del cuaderno anterior que cuando se tenían dos características se utilizó:     

* Tamaño (s)
* Precio (p)

$\sqrt {\frac{s^2+p^2}{2}}$

In [ ]:
def similaridad(obj1, obj2):
  sim = np.sqrt((np.power(obj1,2) + np.power(obj2,2))/2)
  return sim

time1 = np.linspace(start=0, stop=1, num=50)

In [ ]:

sim = similaridad(chocData.Rating,chocData.CocoaPercent)
sns.scatterplot(sim, np.linspace(start=0, stop=1, num=1795))

In [ ]:
#Defina una función para calcular la similaridad de dos objetos
#utilizando el RMSE

In [ ]:
#Calcule la similaridad del primer chocolate con los siguientes 4 ó 5.


Verifique la similaridad obtenida contra su intuición

#4. Usar k-means para agrupar los chocolates, utilice k = 30

In [ ]:
a = np.array(chocData.Rating)
a-


In [ ]:
from sklearn.cluster import KMeans
modelo = KMeans (30)


modelo.fit()

print (modelo.cluster_centers_)

#5. Usar la regla del codo para determinar el número de clusters óptimo